### Vision2Clean AI

In [1]:
# 1️⃣ Install dependencies with CUDA-enabled PyTorch
# Run this once to set up your environment with CUDA support
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --quiet
%pip install ultralytics opencv-python matplotlib --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 2️⃣ Import libraries and check CUDA GPU
import torch
from ultralytics import YOLO
import os

print("=" * 60)
print("🚀 GPU & Environment Check")
print("=" * 60)

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ CUDA version: {torch.version.cuda}")
    print(f"✅ cuDNN version: {torch.backends.cudnn.version()}")
    print(f"✅ GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"   └─ GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"✅ Current device: cuda:{torch.cuda.current_device()}")
    device = "cuda"
else:
    print("⚠️  CUDA not available - will use CPU")
    print("   Run cell 1 to install CUDA-enabled PyTorch")
    device = "cpu"

print(f"\n🎯 Selected device: {device}")
print("=" * 60)

🚀 GPU & Environment Check
✅ PyTorch version: 2.5.1+cu121
✅ CUDA available: True
✅ CUDA version: 12.1
✅ cuDNN version: 90100
✅ GPU count: 1
   └─ GPU 0: NVIDIA GeForce RTX 4050 Laptop GPU
✅ Current device: cuda:0

🎯 Selected device: cuda


In [3]:
# 3️⃣ Define dataset YAML path
data_yaml = r"D:\Documents\Portfolio.github\Vision2Clean-AI\Dataset\data.yaml"

# Check YAML file
!type "{data_yaml}"

train: train/images
val: valid/images
test: test/images

nc: 5
names: ['Cardboard', 'Glass', 'Metal', 'Paper', 'Plastic']

roboflow:
  workspace: waste-segmentation-vlidl
  project: poster-dataset-ctglo
  version: 2
  license: CC BY 4.0
  url: https://universe.roboflow.com/waste-segmentation-vlidl/poster-dataset-ctglo/dataset/2


In [4]:
# 4️⃣ Load pretrained YOLOv11 segmentation model
model = YOLO("yolo11n-seg.pt")

# 5️⃣ Choose device
if torch.backends.mps.is_available():
    device = "mps"     # Apple Silicon GPU
elif torch.cuda.is_available():
    device = "cuda"    # NVIDIA GPU
else:
    device = "cpu"

print(f"🚀 Training on device: {device}")

🚀 Training on device: cuda


---
## 🎓 Training with Checkpoints

**Good news!** YOLO automatically saves your progress:

✅ **Automatic saves:**
- `last.pt` - Latest checkpoint (for resuming)
- `best.pt` - Best model based on validation accuracy
- Saved every 5 epochs + at the end

✅ **You can safely stop training:**
- Press **Interrupt** button or `Ctrl+C`
- All progress up to the last checkpoint is saved
- Resume anytime using Cell 6b

✅ **Checkpoint location:**
```
runs/segment/vision2clean_yolo11_seg/weights/
```

💡 **Pro tip:** If training is too slow, reduce `batch` size or `epochs` in Cell 6.

---

In [5]:
# 🚀 YOLOv11 Segmentation Training with Auto Early-Stopping and Checkpointing

import torch
from ultralytics import YOLO

# Detect device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# Dataset and base model
data_yaml = r"D:\Documents\Portfolio.github\Vision2Clean-AI\Dataset\data.yaml"
model = YOLO("yolo11n-seg.pt")  # Pretrained segmentation model

print("🎯 Starting training on device:", device)
print("📁 Checkpoints will be saved to: runs/segment/vision2clean_yolo11_seg/")
print("💾 Auto-saves: last.pt (resume) & best.pt (best model)")
print("🧠 Early stopping: stops automatically if no improvement for 10 epochs")
print("⏸️ Stop anytime with Ctrl+C - progress is saved!\n")

# Train model with automatic early stopping
model.train(
    data=data_yaml,                # Dataset config
    epochs=100,                    # Max epochs
    imgsz=640,                     # Image size
    batch=8,                       # Adjust based on GPU memory
    device=device,                 # cuda / cpu / mps

    # Saving and project details
    project="runs/segment",
    name="vision2clean_yolo11_seg",
    save=True,
    save_period=5,                 # Save every 5 epochs
    exist_ok=True,                 # Overwrite if same run name

    # Performance
    cache=True,                    # Cache dataset for speed
    workers=4,                     # Data loading threads
    amp=True,                      # Mixed precision training

    # Early stopping
    patience=10,                   # ⛔ Stop if no val improvement in 10 epochs

    # Logging and visualization
    plots=True,                    # Save training plots
    verbose=True                   # Detailed output
)

print("\n✅ Training completed or stopped early based on performance.")
print("🏆 Best model saved as: runs/segment/vision2clean_yolo11_seg/weights/best.pt")
print("💾 Latest checkpoint: runs/segment/vision2clean_yolo11_seg/weights/last.pt")


🎯 Starting training on device: cuda
📁 Checkpoints will be saved to: runs/segment/vision2clean_yolo11_seg/
💾 Auto-saves: last.pt (resume) & best.pt (best model)
🧠 Early stopping: stops automatically if no improvement for 10 epochs
⏸️ Stop anytime with Ctrl+C - progress is saved!

Ultralytics 8.3.214  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Ultralytics 8.3.214  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Documents\Portfolio.github\Vision2Clean-AI\Dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, forma

In [7]:
# 6️⃣b (OPTIONAL) Resume training OR extend completed training
# Run this cell if:
#   - Training was interrupted (will resume from last epoch)
#   - Training completed but you want MORE epochs (will continue training)

import os
import torch
from ultralytics import YOLO

# Re-detect device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"🎯 Device: {device}\n")

checkpoint_path = "runs/segment/vision2clean_yolo11_seg/weights/last.pt"

if os.path.exists(checkpoint_path):
    print(f"✅ Found checkpoint: {checkpoint_path}")
    
    # Load model from checkpoint
    model = YOLO(checkpoint_path)
    
    try:
        # Try to resume (works if training was interrupted)
        print("🔄 Attempting to resume training...\n")
        model.train(resume=True, device=device)
        
    except AssertionError as e:
        if "is finished" in str(e):
            print("⚠️  Training already completed to target epochs!")
            print("\n🔧 Options:")
            print("   1. Model is ready - use Cell 8 to evaluate it")
            print("   2. Train MORE epochs - uncomment and run below:\n")
            print("# Extend training for 50 more epochs:")
            print("# model.train(")
            print("#     data=r'D:\\Documents\\Portfolio.github\\Vision2Clean-AI\\Dataset\\data.yaml',")
            print("#     epochs=50,")
            print("#     device=device,")
            print("#     project='runs/segment',")
            print("#     name='vision2clean_extended'")
            print("# )")
        else:
            raise  # Re-raise if it's a different error
            
else:
    print(f"❌ No checkpoint found at: {checkpoint_path}")
    print("   Run Cell 6 first to start training from scratch")

🎯 Device: cuda

✅ Found checkpoint: runs/segment/vision2clean_yolo11_seg/weights/last.pt
🔄 Attempting to resume training...

Ultralytics 8.3.214  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Documents\Portfolio.github\Vision2Clean-AI\Dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs\segment\vision2clea

In [8]:
# 7️⃣ Evaluate model on validation set
# Run this after training completes to see final metrics

import torch
from ultralytics import YOLO

# Re-detect device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# Load the best trained model
model = YOLO("runs/segment/vision2clean_yolo11_seg/weights/best.pt")

print(f"🎯 Evaluating on device: {device}")
print("=" * 60)

# Run validation
metrics = model.val(device=device)

print("\n✅ Evaluation Results:")
print("=" * 60)
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print("=" * 60)

🎯 Evaluating on device: cuda
Ultralytics 8.3.214  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11n-seg summary (fused): 113 layers, 2,835,543 parameters, 0 gradients, 9.6 GFLOPs
YOLO11n-seg summary (fused): 113 layers, 2,835,543 parameters, 0 gradients, 9.6 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 485.8455.5 MB/s, size: 72.6 KB)
val: Fast image access  (ping: 0.10.1 ms, read: 485.8455.5 MB/s, size: 72.6 KB)
val: Scanning D:\Documents\Portfolio.github\Vision2Clean-AI\Dataset\valid\labels.cache... 626 images, 9 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 626/626  0.0s
val: Scanning D:\Documents\Portfolio.github\Vision2Clean-AI\Dataset\valid\labels.cache... 626 images, 9 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 626/626  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 40/40 4.2it/s 9.4s0.2s
                 Class     Images  I

In [9]:
# 8️⃣ Test on sample image


# 9️⃣ (Optional) Test live webcam
#results = model.predict(source=0, show=True)


In [10]:
# =============================================================
# 🚀 Vision2Clean AI — Test Fine-Tuned YOLOv11-Seg with Webcam
# =============================================================

import cv2
import torch
from ultralytics import YOLO

# 1️⃣ Load your fine-tuned model
model = YOLO("runs/segment/vision2clean_yolo11_seg/weights/best.pt")

# 2️⃣ Choose the best device automatically
if torch.backends.mps.is_available():
    device = "mps"      # Apple Silicon GPU
elif torch.cuda.is_available():
    device = "cuda"     # NVIDIA GPU
else:
    device = "cpu"

print(f"✅ Using device: {device}")

# 3️⃣ Open laptop webcam (0 = default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Could not access webcam.")
    raise SystemExit()

# 4️⃣ Live detection loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv11-Seg inference
    results = model(frame, device=device, verbose=False)[0]

    # Draw masks + labels on the frame
    annotated = results.plot()

    # Display in window
    cv2.imshow("Vision2Clean AI — Live Waste Detection", annotated)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 5️⃣ Cleanup
cap.release()
cv2.destroyAllWindows()
print("👋 Stream closed.")


✅ Using device: cuda
👋 Stream closed.
👋 Stream closed.
